In [8]:
# 1 load xml 2 load jpg 3 haar gray 4 (检测)detect 5 draw(绘制方框)
import cv2
import numpy as np

# 加载load xml 参数1 file name
face_xml = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_xml = cv2.CascadeClassifier('haarcascade_eye.xml')

# load jpg
img = cv2.imread('gr.jpg')
cv2.imshow('src',img)

# haar gray
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

# detect faces 1 data 2 scale 3 5
# 检测人脸: 1 灰度图片数据 2 缩放系数 3 目标大小
faces = face_xml.detectMultiScale(gray,1.3,5)
# 人脸个数
print('face=',len(faces))

# draw方框
for (x,y,w,h) in faces:
    # 起始坐标。起始坐标。
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    # 我们已经找到的人脸区域
    roi_face = gray[y:y+h,x:x+w]
    roi_color = img[y:y+h,x:x+w]
    # 参数1 gray
    eyes = eye_xml.detectMultiScale(roi_face)
    print('eye=',len(eyes))
    for (e_x,e_y,e_w,e_h) in eyes:
        cv2.rectangle(roi_color,(e_x,e_y),(e_x+e_w,e_y+e_h),(0,255,0),2)
cv2.imshow('dst',img)
cv2.waitKey(0)


face= 2
eye= 2
eye= 2


255

### haar特征:
1. 什么是haar？

>特征 = 像素 运算 -> 结果(具体值 向量 矩阵 多维)


2. 如何利用特征 区分目标？ 

>阈值判决 

3. 得到判决？

>机器学习 

- 1 特征 2 判决 3 得到判决

公式推导 权重1为1  权重2为-2

```
特征 = 整个区域*权重1 + 黑色*权重2 = （黑+白）*1+黑*（-2）=
= 黑+白-2黑 = 白-黑
```

- haar模版 从上至下 从左至右 
- image size 模版 size 

```
100*100图片 10*10模板 100次(个模板) step = 10
1. 100*100 2. 10*10 3. step 10 4. 模版1
```

- 模版  不仅可以滑动 还能缩放 `10*10`  第二次滑动 `11*11` 20级(20次缩放)

举例 图片大小`1080*720`  步长step`2` `10*10`模板

```
计算量 = 14个模版*20级缩放*(1080/2*720/2) * (100点+ -) = 50-100亿
(50-100)*15 = 1000亿次
```

## 积分图

![mark](http://myphoto.mtianyan.cn/blog/180327/59AgGbAIfi.png?imageslim)

积分图听起来有点像数据累加。特征计算时讲解了前两种公式。

无论是哪种计算公式，都要求计算矩阵方框中的公式。

所以基于这种思想有人提出了一种简单的算法。

有ABCD四个区域，每个区域都是一个矩形方块。

A区域是左上角那一块小区域，而B区域是包含A区域的长条。

C区域又是包含A的竖直长条。D区域是四个方块之和。

1,2,3,4表明这四个小区域。

进行快速计算:

```
 A 1 B 1 2 C 1 3 D 1 2 3 4
 4 = A-B-C+D = 1+1+2+3+4-1-2-1-3 = 4(10*10 变成3次+-)
```

任意一个方框，可由周围的矩形进行相减得到。

问题: 在计算每一个方块之前，需要将图片上所有的像素全部遍历一次。

![mark](http://myphoto.mtianyan.cn/blog/180327/md68IdC8CC.png?imageslim)

p1 p2 p3 p4 分别是指某一个特征相邻的abcd的模块指针。